In [1]:
# Dataframe
import pandas as pd

# Array
import numpy as np
import ast
import itertools
from tabulate import tabulate
from itertools import islice
from itertools import chain

# STRINGS
import re

import warnings
warnings.filterwarnings("ignore") ## TO AVOID SettingWithCopyWarning

In [3]:
##############################################################
## IMPORT METADATA IN PANDAS
##############################################################
meta_df = pd.read_csv(r'metadata_category_clothing_shoes_and_jewelry_only.csv')
meta_df.head()

,metadataid,asin,salesrank,imurl,categories,title,description,price,related,brand
0,2005401,B00004SR8Z,{'Clothing': 1631},http://ecx.images-amazon.com/images/I/41RfWLMD...,"[['Clothing, Shoes & Jewelry', 'Luggage & Trav...",Lewis N. Clark Deluxe Neck Stash,NaN,12.80,"{'also_bought': ['B004RJWFDU', 'B00920ZC3O', '...",Lewis N. Clark
1,2217897,B0000ZE74A,{'Clothing': 4742},http://ecx.images-amazon.com/images/I/41ryA-RO...,"[['Clothing, Shoes & Jewelry', 'Women', 'Cloth...",Vanity Fair Women's Lollipop Plus Size Cuff Le...,NaN,NaN,"{'also_bought': ['B004PEHJ6U', 'B0000TF5VK', '...",NaN
2,2220611,B00012O12A,{'Clothing': 4},http://ecx.images-amazon.com/images/I/41Dd7rCH...,"[['Clothing, Shoes & Jewelry', 'Men', 'Clothin...",Levi's Men's 505 Regular-Fit Jean,NaN,NaN,"{'also_bought': ['B0018OR118', 'B003M2X1VI', '...",NaN
3,2324985,B0002XSXWC,{'Clothing': 300},http://ecx.images-amazon.com/images/I/41-HS7D7...,"[['Sports & Outdoors', 'Clothing', 'Men', 'Shi...",Columbia Men's Bonehead Short Sleeve Fishing S...,NaN,NaN,"{'also_bought': ['B0031RFWMU', 'B0050VLSLO', '...",NaN
4,2348742,B00066TWMU,{'Clothing': 1199},http://ecx.images-amazon.com/images/I/51UNLUB4...,"[['Clothing, Shoes & Jewelry', 'ASICS'], ['Clo...",ASICS Mesh Backpack,NaN,12.41,"{'also_bought': ['B009DB3RS4', 'B001PFA35G', '...",ASICS


In [4]:
##############################################################
# CHECKING DUPLICATES
##############################################################
meta_df[meta_df.duplicated(['asin'], keep= False)]

,metadataid,asin,salesrank,imurl,categories,title,description,price,related,brand


In [5]:
##############################################################
## CHECKING FOR MISSING VALUES
##############################################################
meta_df.isnull().sum()

metadataid         0
asin               0
salesrank       1107
imurl              0
categories         0
title             23
description    21613
price          12913
related          124
brand          19091
dtype: int64

In [6]:
##############################################################
## DATA TRANSFORMATION - EXPANDING CATEGORIES RANK
##############################################################
file_rank = meta_df[['metadataid','asin', 'salesrank','price']]

def try_literal_eval(e):
    try:
        return ast.literal_eval(e)
    except ValueError:
        return {'No Category': 0}
file_rank['salesrank_fillna'] = pd.DataFrame(file_rank['salesrank'].apply(try_literal_eval))

def split_cat(x):  #### GOT HELP WITH THIS ONE
    if len(x) == 0:
        return {'category': 'No Category', 'rank': 0}
    return  {'category': list(x.keys())[0], 'rank': list(x.values())[0]}
norm_cats = pd.json_normalize(file_rank['salesrank_fillna'].apply(split_cat))

file_rank['category'] = norm_cats['category']
file_rank['rank'] = norm_cats['rank']
file_rank = file_rank[['metadataid','asin','category','rank','price']]
file_rank.head()

,metadataid,asin,category,rank,price
0,2005401,B00004SR8Z,Clothing,1631,12.80
1,2217897,B0000ZE74A,Clothing,4742,NaN
2,2220611,B00012O12A,Clothing,4,NaN
3,2324985,B0002XSXWC,Clothing,300,NaN
4,2348742,B00066TWMU,Clothing,1199,12.41


In [13]:
##############################################################
## DATA TRANSFORMATION - EXPANDING SUB CATEGORIES
##############################################################

file_categories = meta_df[['metadataid','asin','categories']]
file_categories['categories_temp'] = file_categories['categories'].apply(ast.literal_eval).tolist()                                   ## STRING TO LISTS
file_categories['sub_categories_expanded'] = file_categories['categories_temp'].apply(lambda x : set(list(chain.from_iterable(x))))   ## FLATTEN LISTS SUB CATEGORIES AND DEDUPLICATE THEM
file_categories = file_categories.explode('sub_categories_expanded')                                                                  ## EXPANDING SUBCATEGORIES
file_categories_clean = file_categories[['metadataid','asin','sub_categories_expanded']]
file_categories_clean.head(20)
file_categories_clean.to_csv('file_sub_categories.csv')

In [12]:
##############################################################
## DATA TRANSFORMATION - PARSING RELATED
##############################################################
file_related = meta_df[['metadataid','asin', 'related']]
#file_related.head()
def try_literal_eval(e):
    try:
        return ast.literal_eval(e)
    except ValueError:
        return {'also_bought': []}

res = pd.DataFrame(file_related['related'].apply(try_literal_eval).tolist())
file_related_flatten = pd.concat((file_related.drop('related', 1), res), axis=1)
file_related_flatten.head()

,metadataid,asin,also_bought,also_viewed,bought_together,buy_after_viewing
0,2005401,B00004SR8Z,"[B004RJWFDU, B00920ZC3O, B00920ZC3Y, B004RJWFN...","[B00920ZC3Y, B005IUBUKQ, B0054IH8SA, B004RJWFN...",NaN,NaN
1,2217897,B0000ZE74A,"[B004PEHJ6U, B0000TF5VK, B004PEHJ6A, B005JI1U0...","[B004PEHJ6U, B004PEHJ6A, B004A7XXJO, B004PEHJ6...","[B004PEHJ6U, B004A7XXJO, B0007YY8H0, B005JI1U0M]",NaN
2,2220611,B00012O12A,"[B0018OR118, B003M2X1VI, B0030BELHE, B00CRXK0G...","[B0018OR118, B0018OKX68, B0018OT2PG, B003M2X1V...",NaN,NaN
3,2324985,B0002XSXWC,"[B0031RFWMU, B0050VLSLO, B004GJ2G5I, B00A2MSOB...","[B0031RFWMU, B0031RFX7Y, B004GJ2G5I, B00752KFS...","[B0031RFWMU, B0031RFWSE, B0050VLSLO, B004GJ2G5I]",NaN
4,2348742,B00066TWMU,"[B009DB3RS4, B001PFA35G, B00G3K71KS, B00KOWEIA...","[B0030APUDY, B009566FCC, B0047CCO8S, B00J2X1D5...","[B00KOWEIAW, B00G3K71KS, B00CR60HLM, B009566FCC]",NaN


In [22]:
##############################################################
## DATA TRANSFORMATION - EXPLOADING RELATED IN DIFFERENT FILES
##############################################################

### ALSO BOUGHT
file_related_also_bought = file_related_flatten[['metadataid','asin','also_bought']]
file_related_also_bought = file_related_also_bought.explode('also_bought').dropna()
#file_related_also_bought.head()
file_related_also_bought.to_csv('file_related_also_bought.csv')

### ALSO VIEWED
file_related_also_viewed = file_related_flatten[['metadataid','asin','also_viewed']]
file_related_also_viewed = file_related_also_viewed.explode('also_viewed').dropna()
#file_related_also_viewed.head()
file_related_also_viewed.to_csv('file_related_also_viewed.csv')

### BOUGHT TOGETHER
file_related_bought_together = file_related_flatten[['metadataid','asin','bought_together']]
file_related_bought_together = file_related_bought_together.explode('bought_together').dropna()
#file_related_bought_together.head()
file_related_bought_together.to_csv('file_related_bought_together.csv')

### BUY AFTER VIEWING
file_related_buy_after_viewing = file_related_flatten[['metadataid','asin','buy_after_viewing']]
file_related_buy_after_viewing = file_related_buy_after_viewing.explode('buy_after_viewing').dropna()
#file_related_buy_after_viewing.head()
file_related_buy_after_viewing.to_csv('file_related_buy_after_viewing.csv')

In [8]:
##############################################################
## IMPORT PRODUCT REVIEW DATA IN PANDAS TO COMBINE WITH META DATA
##############################################################
review_df_c = pd.read_csv(r'review_df.csv')

In [9]:
##############################################################
## AGGREGATING ON ASIN LEVEL
##############################################################

### REVIEWS FULL DATA
asin_data_all =  review_df_c.groupby('asin',as_index=False).agg({'Rating':'mean',
                                                             'totalVote_score':'mean',
                                                             'wordLength':'mean',
                                                             'sentimentScore':'mean',
                                                             'reviewerID':'count',
                                                             'positiveVote':'sum',
                                                             'totalVote':'sum'
                                                           })
asin_data_all.rename(columns = {'Rating':'avg_Rating_all',
                                'totalVote_score':'avg_totalVote_score_all_incorrect',
                                'wordLength':'avg_wordLength_all',
                                'sentimentScore':'avg_sentimentScore_all',
                                'reviewerID':'cnt_Reviews_all',
                                'positiveVote':'sum_positiveVote_all',
                                'totalVote':'sum_totalVote_all'
                               }, inplace = True)

### REVIEWS ONLY WITH VOTES DATA
asin_data_votes =  review_df_c[review_df_c['totalVote_flag'] == 'Votes'].groupby('asin',as_index=False).agg({'Rating':'mean',
                                                                                                             'totalVote_score':'mean',
                                                                                                             'wordLength':'mean',
                                                                                                             'sentimentScore':'mean',
                                                                                                             'reviewerID':'count'
                                                                                                           })
asin_data_votes.rename(columns = {'Rating':'avg_Rating_votes',
                                  'totalVote_score':'avg_totalVote_score_votes_correct',
                                  'wordLength':'avg_wordLength_votes',
                                  'sentimentScore':'avg_sentimentScore_votes',
                                  'reviewerID':'cnt_Reviews_votes'
                                 }, inplace = True)

### REVIEWS ONLY WITH SENTIMENTS DATA
### POSITIVE SENTIMENT
asin_data_pos_sen =  review_df_c[review_df_c['sentimentScore_flag'] == 'pos'].groupby('asin',as_index=False).agg({'Rating':'mean',
                                                                                                                  'totalVote_score':'mean',
                                                                                                                  'wordLength':'mean',
                                                                                                                  'sentimentScore':'mean',
                                                                                                                  'reviewerID':'count'
                                                                                                                 })
asin_data_pos_sen.rename(columns = {'Rating':'avg_Rating_pos_sen',
                                    'totalVote_score':'avg_totalVote_score_votes_pos_sen',
                                    'wordLength':'avg_wordLength_pos_sen',
                                    'sentimentScore':'avg_sentimentScore_pos_sen',
                                    'reviewerID':'cnt_Reviews_pos_sen'
                                   }, inplace = True)
### NEUTRAL SENTIMENT
asin_data_neu_sen =  review_df_c[review_df_c['sentimentScore_flag'] == 'neu'].groupby('asin',as_index=False).agg({'Rating':'mean',
                                                                                                                  'totalVote_score':'mean',
                                                                                                                  'wordLength':'mean',
                                                                                                                  'sentimentScore':'mean',
                                                                                                                  'reviewerID':'count'
                                                                                                                 })
asin_data_neu_sen.rename(columns = {'Rating':'avg_Rating_neu_sen',
                                    'totalVote_score':'avg_totalVote_score_votes_neu_sen',
                                    'wordLength':'avg_wordLength_neu_sen',
                                    'sentimentScore':'avg_sentimentScore_neu_sen',
                                    'reviewerID':'cnt_Reviews_neu_sen'
                                   }, inplace = True)
### NEGATIVE SENTIMENT
asin_data_neg_sen =  review_df_c[review_df_c['sentimentScore_flag'] == 'neg'].groupby('asin',as_index=False).agg({'Rating':'mean',
                                                                                                                  'totalVote_score':'mean',
                                                                                                                  'wordLength':'mean',
                                                                                                                  'sentimentScore':'mean',
                                                                                                                  'reviewerID':'count'
                                                                                                                 })
asin_data_neg_sen.rename(columns = {'Rating':'avg_Rating_neg_sen',
                                    'totalVote_score':'avg_totalVote_score_votes_neg_sen',
                                    'wordLength':'avg_wordLength_neg_sen',
                                    'sentimentScore':'avg_sentimentScore_neg_sen',
                                    'reviewerID':'cnt_Reviews_neg_sen'
                                   }, inplace = True)


### COMBINE REVIEWS AND META DATA ON ASIN LEVEL
asin_data = pd.merge(file_rank, asin_data_all, on='asin', how='left')                     
asin_data = pd.merge(asin_data, asin_data_votes, on='asin', how='left')
asin_data = pd.merge(asin_data, asin_data_pos_sen, on='asin', how='left')
asin_data = pd.merge(asin_data, asin_data_neu_sen, on='asin', how='left')
asin_data = pd.merge(asin_data, asin_data_neg_sen, on='asin', how='left')
#asin_data.head()
asin_data.to_csv('asin_df.csv')

In [23]:
##############################################################
## IMPORT AGGREGATIED ON ASIN LEVEL DATA
##############################################################
asin_df = pd.read_csv(r'asin_df.csv')
asin_df.head()

,Unnamed: 0,metadataid,asin,category,rank,price,avg_Rating_all,avg_totalVote_score_all_incorrect,avg_wordLength_all,avg_sentimentScore_all,...,avg_Rating_neu_sen,avg_totalVote_score_votes_neu_sen,avg_wordLength_neu_sen,avg_sentimentScore_neu_sen,cnt_Reviews_neu_sen,avg_Rating_neg_sen,avg_totalVote_score_votes_neg_sen,avg_wordLength_neg_sen,avg_sentimentScore_neg_sen,cnt_Reviews_neg_sen
0,0,2005401,B00004SR8Z,Clothing,1631,12.80,4.545455,29.063361,62.681818,0.557045,...,4.833333,33.333333,35.500000,0.234033,6.0,4.000000,0.000000,61.0,-0.927800,1.0
1,1,2217897,B0000ZE74A,Clothing,4742,NaN,4.363636,9.090909,48.818182,0.455409,...,3.333333,0.000000,43.666667,-0.010800,3.0,4.000000,0.000000,171.0,-0.852000,1.0
2,2,2220611,B00012O12A,Clothing,4,NaN,4.153439,13.492063,58.687831,0.601805,...,3.777778,11.111111,50.333333,0.182911,45.0,2.909091,27.272727,84.0,-0.670164,11.0
3,3,2324985,B0002XSXWC,Clothing,300,NaN,4.578947,10.165904,42.394737,0.614816,...,4.300000,24.130435,42.700000,0.104180,10.0,5.000000,0.000000,20.0,-0.726300,1.0
4,4,2348742,B00066TWMU,Clothing,1199,12.41,3.857143,14.285714,50.285714,0.596864,...,2.500000,50.000000,38.500000,0.195950,2.0,1.000000,100.000000,22.0,-0.695600,1.0
